<a href="https://colab.research.google.com/github/osullik/bc-autoreporter/blob/main/Summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [120]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [121]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [122]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [123]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [124]:
import torch

device = torch.device("cpu")

if torch.cuda.is_available():
   print("Training on GPU")
   device = torch.device("cuda:0")

Training on GPU


In [125]:
import random
import re
import numpy as np

# Read in data
Assume each set of evaluations corresponds to the same employee. Different employees will be read in separately and run through the model separately to generate per-employee summaries.

In [ ]:
# Gather data to summarize
original_text = "Junk foods taste good that’s why it is mostly liked by everyone of any age group especially kids and school going children. They generally ask for the junk food daily because they have been trend so by their parents from the childhood. They never have been discussed by their parents about the harmful effects of junk foods over health. According to the research by scientists, it has been found that junk foods have negative effects on the health in many ways. They are generally fried food found in the market in the packets. They become high in calories, high in cholesterol, low in healthy nutrients, high in sodium mineral, high in sugar, starch, unhealthy fat, lack of protein and lack of dietary fibers. Processed and junk foods are the means of rapid and unhealthy weight gain and negatively impact the whole body throughout the life. It makes able a person to gain excessive weight which is called as obesity. Junk foods tastes good and looks good however do not fulfil the healthy calorie requirement of the body. Some of the foods like french fries, fried foods, pizza, burgers, candy, soft drinks, baked goods, ice cream, cookies, etc are the example of high-sugar and high-fat containing foods. It is found according to the Centres for Disease Control and Prevention that Kids and children eating junk food are more prone to the type-2 diabetes. In type-2 diabetes our body become unable to regulate blood sugar level. Risk of getting this disease is increasing as one become more obese or overweight. It increases the risk of kidney failure. Eating junk food daily lead us to the nutritional deficiencies in the body because it is lack of essential nutrients, vitamins, iron, minerals and dietary fibers. It increases risk of cardiovascular diseases because it is rich in saturated fat, sodium and bad cholesterol. High sodium and bad cholesterol diet increases blood pressure and overloads the heart functioning. One who like junk food develop more risk to put on extra weight and become fatter and unhealthier. Junk foods contain high level carbohydrate which spike blood sugar level and make person more lethargic, sleepy and less active and alert. Reflexes and senses of the people eating this food become dull day by day thus they live more sedentary life. Junk foods are the source of constipation and other disease like diabetes, heart ailments, clogged arteries, heart attack, strokes, etc because of being poor in nutrition. Junk food is the easiest way to gain unhealthy weight. The amount of fats and sugar in the food makes you gain weight rapidly. However, this is not a healthy weight. It is more of fats and cholesterol which will have a harmful impact on your health. Junk food is also one of the main reasons for the increase in obesity nowadays.This food only looks and tastes good, other than that, it has no positive points. The amount of calorie your body requires to stay fit is not fulfilled by this food. For instance, foods like French fries, burgers, candy, and cookies, all have high amounts of sugar and fats. Therefore, this can result in long-term illnesses like diabetes and high blood pressure. This may also result in kidney failure. Above all, you can get various nutritional deficiencies when you don’t consume the essential nutrients, vitamins, minerals and more. You become prone to cardiovascular diseases due to the consumption of bad cholesterol and fat plus sodium. In other words, all this interferes with the functioning of your heart. Furthermore, junk food contains a higher level of carbohydrates. It will instantly spike your blood sugar levels. This will result in lethargy, inactiveness, and sleepiness. A person reflex becomes dull overtime and they lead an inactive life. To make things worse, junk food also clogs your arteries and increases the risk of a heart attack. Therefore, it must be avoided at the first instance to save your life from becoming ruined.The main problem with junk food is that people don’t realize its ill effects now. When the time comes, it is too late. Most importantly, the issue is that it does not impact you instantly. It works on your overtime; you will face the consequences sooner or later. Thus, it is better to stop now.You can avoid junk food by encouraging your children from an early age to eat green vegetables. Their taste buds must be developed as such that they find healthy food tasty. Moreover, try to mix things up. Do not serve the same green vegetable daily in the same style. Incorporate different types of healthy food in their diet following different recipes. This will help them to try foods at home rather than being attracted to junk food.In short, do not deprive them completely of it as that will not help. Children will find one way or the other to have it. Make sure you give them junk food in limited quantities and at healthy periods of time. "

In [ ]:
original_text1 = "On January 15th, @WaylonSmithers was observed to perform to an excellent standard. This was evidenced by his meticulous preparation and coordination of a company-wide charity event, ensuring its success. His actions show #meticulousness and #precision."
original_text2 = "On April 10th, @WaylonSmithers was observed to perform to an excellent standard. This was evidenced by his exceptional leadership and direction of a complex merger between two companies, ensuring its success. His actions show #leadership and #vision."
original_text3 = "On July 4th, @WaylonSmithers was observed to perform to an excellent standard. This was evidenced by his unwavering dedication to perfecting the design of a new product line, ensuring its success in the market. His actions show #dedication and #discipline."
original_text4 = "On October 31st, @WaylonSmithers was observed to perform to an excellent standard. This was evidenced by his outstanding initiative in organizing a successful product launch event, garnering media attention and generating high sales. His actions show #creativity and #organizationalSkills."

In [ ]:
original_text = original_text1 + ' ' + original_text2 + ' ' + original_text3 + ' ' + original_text4
original_text

NameError: ignored

In [126]:
original_text = "On January 15th, 2021, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his successful completion of a routine safety check on Reactor 4. His actions show #competence #attentiontodetail #safetyconsciousness. On February 2nd, 2021, @HomerSimpson was observed to perform to a poor standard. This was evidenced by his failure to properly label hazardous waste containers in the storage room, leading to a safety hazard. His actions show #carelessness #lackofattention #safetyoversight. On March 12th, 2021, @HomerSimpson was observed to perform to an excellent standard. This was evidenced by his quick thinking and calm demeanor during an unexpected power surge in the control room. His actions show #resilience #problem-solving #teamwork.On April 5th, 2021, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his leadership during a fire drill, guiding his team to safety and ensuring that all necessary procedures were followed. His actions show #leadership #teamcoordination #safetyconsciousness. On May 22nd, 2021, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his successful completion of a routine maintenance check on the cooling system, though he did require some prompting to remember certain procedures. His actions show #competence #teachable #attentiontodetail. On June 7th, 2021, @HomerSimpson was observed to perform to a poor standard. This was evidenced by his failure to report a leak in a coolant pipe, which caused damage to nearby equipment. His actions show #negligence #lackofresponsibility #safetyoversight. On July 15th, 2021, @HomerSimpson was observed to perform to a good standard. This was evidenced by his successful completion of a new training module on emergency response procedures. His actions show #willingnesstolearn #adaptability #safetyconsciousness. On August 9th, 2021, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his coordination of a team to repair a malfunctioning steam valve, minimizing downtime and avoiding any safety hazards. His actions show #teamcoordination #problem-solving #efficiency. On September 27th, 2021, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his successful completion of a routine inspection of the turbine generators, though he did forget to record one measurement. His actions show #competence #attentiontodetail #teachable. On October 14th, 2021, @HomerSimpson was observed to perform to an excellent standard. This was evidenced by his quick response to a power outage in Sector 7G, restoring power and ensuring that critical systems remained operational. His actions show #resilience #problem-solving #safetyconsciousness. On May 3rd @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his willingness to work overtime to assist with the cleanup of the oil spill. His actions show #dedication and #cooperativeness. On May 11th @HomerSimpson was observed to perform to a poor standard. This was evidenced by his careless handling of nuclear waste, which resulted in a spill that contaminated the town's water supply. His actions show #recklessness and #carelessness. On May 27th @HomerSimpson was observed to perform to a poor standard. This was evidenced by his lack of concern for the safety of his fellow workers when he removed safety equipment in order to increase his productivity. His actions show #irresponsibility and #selfishness. On June 3rd @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his ability to work effectively under pressure when the plant's emergency systems were activated. His actions show #calmness and #levelheadedness. On June 9th @HomerSimpson was observed to perform to a good standard. This was evidenced by his creative solution to the plant's budget problems, which involved turning off unnecessary equipment. His actions show #innovation and #costconsciousness. On June 16th @HomerSimpson was observed to perform to a very good standard. This was evidenced by his success in negotiating a union contract that was fair to both the workers and the company. His actions show #diplomacy and #negotiation skills. On June 23rd @HomerSimpson was observed to perform to a poor standard. This was evidenced by his careless use of the company car, which resulted in damage to both the car and the property of others. His actions show #carelessness and #negligence. On June 29th @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his willingness to work on a holiday in order to assist with an important project. His actions show #dedication and #teamwork. On August 1st, while filling in for Smithers, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his organization of Mr. Burns' schedule and preparation of his meals. His actions show #responsibility, #efficiency, and #attentiontodetail. On August 3rd, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his handling of Mr. Burns' incoming mail and scheduling of appointments. His actions show #competence and #accuracy, but there is room for improvement. On August 7th, @HomerSimpson was observed to perform to a poor standard. This was evidenced by his mishandling of Mr. Burns' laundry, resulting in several ruined garments. His actions show #carelessness and #inattention. On August 10th, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his creative solution to a scheduling conflict, allowing Mr. Burns to attend both a charity event and a business meeting. His actions show #resourcefulness and #innovation. On August 15th, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his coordination of Mr. Burns' transportation for the day. His actions show #adequacy and #dependability, but once again, there is room for improvement. On August 18th, @HomerSimpson was observed to perform to a good standard. This was evidenced by his successful negotiation of a contract with a new supplier, resulting in cost savings for the company. His actions show #assertiveness and #businesssavvy. On August 22nd, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his management of Mr. Burns' finances, including payment of bills and tracking of expenses. His actions show #financialacumen and #thoroughness. On August 25th, @HomerSimpson was observed to perform to a poor standard. This was evidenced by his forgetting to inform Mr. Burns of an important meeting, resulting in a missed opportunity for the company. His actions show #forgetfulness and #lackofattention. On August 28th, @HomerSimpson was observed to perform to a good standard. This was evidenced by his troubleshooting of a technical issue with the office equipment, avoiding a costly repair bill. His actions show #technicalskill and #problem-solving. On August 31st, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his handling of Mr. Burns' correspondence and phone calls for the day. His actions show #mediocrity and #routine, but no major mistakes were made."

# Preprocessing Function

In [ ]:
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ', text)
    text = re.sub(r" +", ' ', text)
    return text

In [127]:
# generate chunks of text \ sentences <= 512 tokens
def nest_sentences(document):
  nested = []
  sent = ["summarize"]
  length = 0
  for sentence in nltk.sent_tokenize(document):
    #sentence = 'summarize ' + sentence
    print(sentence)
    length += len(sentence)
    if length < 512:
      sent.append(sentence)
    else:
      nested.append(sent)
      sent = []
      length = 0

  if sent:
    random.shuffle(sent)
    result = ''.join(sent)
    nested.append(result)
  return nested

In [ ]:
# generate chunks of text \ sentences <= 512 tokens
def shuffle_sentences(document):
  x = document.split('.')
  random.shuffle(x)
  shuffled = ' '.join(x)
  return "summarize: " + shuffled 

In [ ]:
shuffle_sentences(original_text)

"summarize:  This was evidenced by his quick thinking and calm demeanor during an unexpected power surge in the control room  Burns' finances, including payment of bills and tracking of expenses  This was evidenced by his leadership during a fire drill, guiding his team to safety and ensuring that all necessary procedures were followed  Burns' laundry, resulting in several ruined garments  On July 15th, 2021, @HomerSimpson was observed to perform to a good standard  On September 27th, 2021, @HomerSimpson was observed to perform to a satisfactory standard  This was evidenced by his mishandling of Mr  On May 27th @HomerSimpson was observed to perform to a poor standard  Burns' schedule and preparation of his meals  On October 14th, 2021, @HomerSimpson was observed to perform to an excellent standard  Burns of an important meeting, resulting in a missed opportunity for the company  On August 7th, @HomerSimpson was observed to perform to a poor standard  This was evidenced by his coordinat

# Summarization function

In [128]:
# generate summary on text with <= 512 tokens
def generate_summary_nested(nested_sentences, tokenizer, model):
  summaries = []
  for nested in nested_sentences:
    print(nested)
    input_tokenized = tokenizer.encode(' '.join(nested), truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = model.to(device).generate(input_tokenized,
                                            num_beams=4,
                                            no_repeat_ngram_size=2,
                                            min_length=5,
                                            max_length=100,
                                            early_stopping=True)
    output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

In [137]:
# generate summary on text with <= 512 tokens
def generate_summary(sentences, tokenizer, model):
  input_tokenized = tokenizer.encode(sentences, truncation=True, return_tensors='pt')
  print(input_tokenized)
  input_tokenized = input_tokenized.to(device)
  summary_ids = model.to(device).generate(input_tokenized,
                                          num_beams=4,
                                          min_length=50,
                                          max_length=1000,
                                          early_stopping=True)
  output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  return output

In [150]:
' '.join(inputs_squeezed)

'summarize  On  January  15th,  2021,  @HomerSimpson  was  observed  to  perform  to  a  satisfactory  standard.  This  was  evidenced  by  his  successful  completion  of  a  routine  safety  check  on  Reactor  4.  His  actions  show  #competence  #attentiontodetail  #safetyconsciousness.  On  February  2nd,  2021,  @HomerSimpson  was  observed  to  perform  to  a  poor  standard.  This  was  evidenced  by  his  failure  to  properly  label  hazardous  waste  containers  in  the  storage  room,  leading  to  a  safety  hazard. '

In [151]:
generate_summary(' '.join(inputs_squeezed), tokenizer, model)

tensor([[21603,   461,  1762,   627,   189,     6,   460,  2658,     6,  3320,
         19040,    52,   134,    23,  1167,   739,    47,  6970,    12,  1912,
            12,     3,     9, 25140,  1068,     5,   100,    47,  2084,    26,
            57,   112,  1574,  6929,    13,     3,     9,  3860,  1455,   691,
            30,   419,     9,  5317,  2853,   978,  2874,   504,  1713, 30916,
          3772,  1713, 25615,   235,   221,  5756,  1713, 15233,    17,    63,
         25098,   655,     5,   461,  2083,   204,   727,     6,   460,  2658,
             6,  3320, 19040,    52,   134,    23,  1167,   739,    47,  6970,
            12,  1912,    12,     3,     9,  2714,  1068,     5,   100,    47,
          2084,    26,    57,   112,  3338,    12,  3085,  3783, 20793,  2670,
         10827,    16,     8,  1606,   562,     6,  1374,    12,     3,     9,
          1455,     3, 10557,   986,     5,     1]])


['@HomerSimpson was observed to perform to a satisfactory standard . his actions show #competence #attentiontodetail #safetyconsciousness . this was evidenced by his failure to properly label hazardous waste containers in the storage room .']

In [ ]:
# from https://github.com/saarthdeshpande/book-summarizer/blob/master/model.py
def generate_summary(text, tokenizer, model):
    preprocess_text = text.strip().replace("\n", "")
    t5_prepared_Text = "summarize: " + preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, truncation=True, return_tensors="pt")
    print("Num toks = ", len(preprocess_text))
    tokenized_text = tokenized_text.to(device)

    summary_ids = model.generate(tokenized_text,
                                 num_beams=5,
                                 no_repeat_ngram_size=2,
                                 min_length=30,
                                 max_length=96,
                                 early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

# Create Tokenizer
Corresponding T5 tokenizer from Hugging Face library

In [130]:
# Instantiate the tokenizer 
tokenizer = T5Tokenizer.from_pretrained('t5-small')
##tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
print(tokenizer.model_max_length)

512


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [131]:
# # Concatenate the word "summarize:" to raw text to tell the model what task to perform on the input
# preprocess_text = original_text.strip().replace("\n","")
# t5_prepared_Text = "summarize: "+preprocess_text
# print ("original text preprocessed: \n", preprocess_text)

# tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt")

# Create Model
Pretrained T5 model from Hugging Face library. This is a generic model that can do translation, summarization, etc. We will pass it the 'summarize' command appended to the front of the input stream to trigger a summary output from the model.

https://towardsdatascience.com/simple-abstractive-text-summarization-with-pretrained-t5-text-to-text-transfer-transformer-10f6d602c426 

In [132]:
# Instantiate the model 
model = T5ForConditionalGeneration.from_pretrained('t5-small')
##model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)

# Summarize
Generate summary text given preprocessed input text containing a set of reports for a given employee.

In [ ]:
# Preprocess text into chunks based on model max length
cleaned_text = clean_text(original_text)
#shuffled_sentences = shuffle_sentences(cleaned_text)

In [146]:
nested_text = nest_sentences(original_text)
inputs = []
for i in nested_text[0]:
  inputs.append(i.split())  # TODO loop over all the chunks to get a long summar for each employee
inputs_squeezed = np.concatenate( inputs, axis=0 )
inputs_squeezed = [i + ' ' for i in inputs_squeezed]

On January 15th, 2021, @HomerSimpson was observed to perform to a satisfactory standard.
This was evidenced by his successful completion of a routine safety check on Reactor 4.
His actions show #competence #attentiontodetail #safetyconsciousness.
On February 2nd, 2021, @HomerSimpson was observed to perform to a poor standard.
This was evidenced by his failure to properly label hazardous waste containers in the storage room, leading to a safety hazard.
His actions show #carelessness #lackofattention #safetyoversight.
On March 12th, 2021, @HomerSimpson was observed to perform to an excellent standard.
This was evidenced by his quick thinking and calm demeanor during an unexpected power surge in the control room.
His actions show #resilience #problem-solving #teamwork.On April 5th, 2021, @HomerSimpson was observed to perform to a very good standard.
This was evidenced by his leadership during a fire drill, guiding his team to safety and ensuring that all necessary procedures were followed

In [141]:
# Summarize preprocessed text
generate_summary(inputs_squeezed, tokenizer, model)

ValueError: ignored

In [ ]:
# Summarize preprocessed text
#generate_summary(shuffled_sentences, tokenizer, model)

In [ ]:
shuffled_sentences

[' This was evidenced by his quick thinking and calm demeanor during an unexpected power surge in the control room',
 ' On August 10th, @HomerSimpson was observed to perform to a very good standard',
 ' On August 18th, @HomerSimpson was observed to perform to a good standard',
 ' This was evidenced by his creative solution to a scheduling conflict, allowing Mr',
 'On April 5th, 2021, @HomerSimpson was observed to perform to a very good standard',
 ' On February 2nd, 2021, @HomerSimpson was observed to perform to a poor standard',
 ' His actions show #responsibility, #efficiency, and #attentiontodetail',
 ' On August 9th, 2021, @HomerSimpson was observed to perform to a very good standard',
 ' This was evidenced by his successful completion of a new training module on emergency response procedures',
 ' On March 12th, 2021, @HomerSimpson was observed to perform to an excellent standard',
 ' His actions show #willingnesstolearn #adaptability #safetyconsciousness',
 ' This was evidenced by

In [ ]:
# # summmarize 
# summary_ids = model.generate(tokenized_text,
#                                     num_beams=4,
#                                     no_repeat_ngram_size=2,
#                                     min_length=500,
#                                     max_length=1000,
#                                     early_stopping=True)

# output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# print ("\n\nSummarized text: \n",output)



Summarized text: 
 @HomerSimpson was observed to perform to a satisfactory standard. this was evidenced by his failure to properly label hazardous waste containers in the storage room, leading to safety hazard. his actions show #resilience #problem-solving #safetyoversight. on may 27th, 2021, @homersimpedson performed to an excellent standard. he did forget to record one measurement, resulting in several ruined garments, and his efforts show'responsibility........................................ in dr st. pep'd o e w t y, the! &, in an un ned,,' ; i m com u.s. and re, his action show self-help, on august, at the end? raf was seen if the company's own – - and not to the safety of l', an alleged thoroughness / and #sys and the same : b p f, to his en, her, or the, that ’s to have c., #selfishness and/ or an, but.... to get his work despite wards vs the in his, "the......... and... on g z x and other %, this â€” his own,......and..... and, of, just ten ng, with the work,#self — ­,’ ing 